<a href="https://colab.research.google.com/github/janaghoniem/Social-Media-Sentiment-Analysis/blob/main/ML_Arabic%26English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from scipy.sparse import vstack
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

### Arabic Testing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/embedded_arabic.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,text,stemmedtext,label,text_embeddings
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,متز نوع ما . نظف وقع جهز شاطيء . طعم,2,[-8.53703637e-03 -1.90370362e-02 1.65396303e-...
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,احد سبب نجح امر ان شخص دول عشق ترب . نحب امر ....,2,[-3.13104391e-02 -4.76208851e-02 1.56378314e-...
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,هدف . وقي . نقل صخب شرع قهر الي هدء جبل شيش . ...,2,[-3.92383114e-02 -4.87467274e-02 1.76576644e-...
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,خلص . بدي الل مست بهر زي فيل زرق ميقراش احس . ...,2,[-0.01965529 -0.05666145 0.15446861 -0.006716...
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,ياس جلر جزء لا تجز دبي . ندق كامل خدم ريح نفس ...,2,[-2.29525026e-02 -5.09175062e-02 1.79437503e-...


In [ ]:
!pip install --upgrade numpy
!pip uninstall -y gensim
!pip install gensim


  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metada

1) Random forest

In [8]:
import pandas as pd
import numpy as np
import ast
import re
import pickle
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
# import gensim.downloader as api

from google.colab import drive
drive.mount('/content/drive')
with open("/content/drive/MyDrive/Colab Notebooks/Data Science/Arabic Sentiment/Features/training_data.pkl", 'rb') as f:
    training_data = pickle.load(f)

X = training_data['X_fasttext']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
y_cat = tf.keras.utils.to_categorical(y)

X_temp, X_test, y_temp, y_test = train_test_split(X, y_cat, test_size=0.2, stratify=y_cat, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# Train initial Random Forest
rf_model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate
print("Initial Random Forest Training Accuracy:", accuracy_score(y_train, rf_model.predict(X_train)))
print("Initial Random Forest Validation Accuracy:", accuracy_score(y_val, rf_model.predict(X_val)))
print("Initial Random Forest Testing Accuracy:", accuracy_score(y_test, rf_model.predict(X_test)))

print("--------------------------------------------------")

# Combine train and validation sets for hyperparameter tuning
X_train_val = np.vstack([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

# Define hyperparameter grid
rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid Search with Cross Validation
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42),
                       rf_params,
                       cv=3,
                       n_jobs=-1,
                       verbose=1)
rf_grid.fit(X_train_val, y_train_val)

# Best model
best_rf = rf_grid.best_estimator_

# Evaluate best model
print("Best Parameters:", rf_grid.best_params_)
print("Best Random Forest Training Accuracy:", accuracy_score(y_train, best_rf.predict(X_train)))
print("Best Random Forest Validation Accuracy:", accuracy_score(y_val, best_rf.predict(X_val)))
print("Best Random Forest Testing Accuracy:", accuracy_score(y_test, best_rf.predict(X_test)))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Data Science/Arabic Sentiment/Features/training_data.pkl'

2) SVM

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(X)

# First split: 80% train/valid, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Second split: 80% train, 20% validation from the 80% temp data
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 of 80% = 20%

# SVM Classifier
svm_model = LinearSVC()
svm_model.fit(X_train, y_train)

# Predictions
train_preds = svm_model.predict(X_train)
val_preds = svm_model.predict(X_val)
test_preds = svm_model.predict(X_test)

# Accuracy scores
train_accuracy = accuracy_score(y_train, train_preds)
validation_accuracy = accuracy_score(y_val, val_preds)
test_accuracy = accuracy_score(y_test, test_preds)

# Output
print("SVM Training Accuracy:", train_accuracy)
print("SVM Validation Accuracy:", validation_accuracy)
print("SVM Testing Accuracy:", test_accuracy)

# Common setup for all models
X_temp, X_test, y_temp, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Combine train and validation sets for GridSearchCV
X_train_val = vstack([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

# SVM Hyperparameter Tuning
# Improved SVM Hyperparameter Tuning
print("\nStarting SVM hyperparameter tuning...")
svm_params = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2'],  # Only l2 penalty to avoid incompatible combinations
    'loss': ['squared_hinge'],  # Only squared_hinge to avoid issues
    'dual': [False]  # Prefer primal form for large n_samples > n_features
}

svm_grid = GridSearchCV(
    LinearSVC(random_state=42, max_iter=10000),  # Increased max_iter for convergence
    svm_params,
    cv=3,
    n_jobs=-1,
    verbose=1
)
svm_grid.fit(X_train_val, y_train_val)
best_svm = svm_grid.best_estimator_

# Predictions
svm_train_preds = best_svm.predict(X_train)
svm_val_preds = best_svm.predict(X_val)
svm_test_preds = best_svm.predict(X_test)

# Accuracy scores
svm_train_acc = accuracy_score(y_train, svm_train_preds)
svm_val_acc = accuracy_score(y_val, svm_val_preds)
svm_test_acc = accuracy_score(y_test, svm_test_preds)

print("SVM - Best Parameters:", svm_grid.best_params_)
print("SVM Training Accuracy:", svm_train_acc)
print("SVM Validation Accuracy:", svm_val_acc)
print("SVM Testing Accuracy:", svm_test_acc)
print("----------------------------------------")

3) Logistic regression

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(X)

# First split: 80% train/valid, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Second split: 80% train, 20% validation from the 80% temp data
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 of 80%

# Logistic Regression Model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

# Predictions
train_preds = lr_model.predict(X_train)
val_preds = lr_model.predict(X_val)
test_preds = lr_model.predict(X_test)

# Accuracy scores
train_accuracy = accuracy_score(y_train, train_preds)
validation_accuracy = accuracy_score(y_val, val_preds)
test_accuracy = accuracy_score(y_test, test_preds)

# Output
print("Logistic Regression Training Accuracy:", train_accuracy)
print("Logistic Regression Validation Accuracy:", validation_accuracy)
print("Logistic Regression Testing Accuracy:", test_accuracy)

# Common setup for all models
X_temp, X_test, y_temp, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Combine train and validation sets for GridSearchCV
X_train_val = vstack([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

# Logistic Regression Hyperparameter Tuning
lr_params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

lr_grid = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42),
                      lr_params,
                      cv=3,
                      n_jobs=-1,
                      verbose=1)
lr_grid.fit(X_train_val, y_train_val)

# Best model
best_lr = lr_grid.best_estimator_

# Predictions
lr_train_preds = best_lr.predict(X_train)
lr_val_preds = best_lr.predict(X_val)
lr_test_preds = best_lr.predict(X_test)

# Accuracy scores
lr_train_acc = accuracy_score(y_train, lr_train_preds)
lr_val_acc = accuracy_score(y_val, lr_val_preds)
lr_test_acc = accuracy_score(y_test, lr_test_preds)

print("Logistic Regression - Best Parameters:", lr_grid.best_params_)
print("Logistic Regression Training Accuracy:", lr_train_acc)
print("Logistic Regression Validation Accuracy:", lr_val_acc)
print("Logistic Regression Testing Accuracy:", lr_test_acc)
print("----------------------------------------")

4) Naive Bayes

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(X)

# First split: 80% train/valid, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Second split: 80% train, 20% validation from the 80% temp data
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 of 80%

# Naive Bayes Model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predictions
train_preds = nb_model.predict(X_train)
val_preds = nb_model.predict(X_val)
test_preds = nb_model.predict(X_test)

# Accuracy scores
train_accuracy = accuracy_score(y_train, train_preds)
validation_accuracy = accuracy_score(y_val, val_preds)
test_accuracy = accuracy_score(y_test, test_preds)

# Output
print("Naive Bayes Training Accuracy:", train_accuracy)
print("Naive Bayes Validation Accuracy:", validation_accuracy)
print("Naive Bayes Testing Accuracy:", test_accuracy)

# Common setup for all models
X_temp, X_test, y_temp, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Combine train and validation sets for GridSearchCV
X_train_val = vstack([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

# Naive Bayes Hyperparameter Tuning
nb_params = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
    'fit_prior': [True, False]
}

nb_grid = GridSearchCV(MultinomialNB(),
                      nb_params,
                      cv=3,
                      n_jobs=-1,
                      verbose=1)
nb_grid.fit(X_train_val, y_train_val)

# Best model
best_nb = nb_grid.best_estimator_

# Predictions
nb_train_preds = best_nb.predict(X_train)
nb_val_preds = best_nb.predict(X_val)
nb_test_preds = best_nb.predict(X_test)

# Accuracy scores
nb_train_acc = accuracy_score(y_train, nb_train_preds)
nb_val_acc = accuracy_score(y_val, nb_val_preds)
nb_test_acc = accuracy_score(y_test, nb_test_preds)

print("Naive Bayes - Best Parameters:", nb_grid.best_params_)
print("Naive Bayes Training Accuracy:", nb_train_acc)
print("Naive Bayes Validation Accuracy:", nb_val_acc)
print("Naive Bayes Testing Accuracy:", nb_test_acc)
print("----------------------------------------")

### English testing

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/English_cleaned.csv')
X = df['normalized_tweet']
y = df['target']

1) Random Forest

In [ ]:
# Split into train+validation and test sets first (80% train_val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Then split train+validation into train and validation sets (80/20 split of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)  # 0.25 x 0.8 = 0.2, so final: 60% train, 20% val, 20% test

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Random Forest Classifier
rf_model = RandomForestClassifier(
    n_estimators=100,  # Number of trees
    random_state=42,
    n_jobs=-1  # Use all cores for training
)
rf_model.fit(X_train_tfidf, y_train)

# Predictions
y_train_pred = rf_model.predict(X_train_tfidf)
y_val_pred = rf_model.predict(X_val_tfidf)
y_test_pred = rf_model.predict(X_test_tfidf)

# Accuracy calculations
train_acc = accuracy_score(y_train, y_train_pred)
val_acc = accuracy_score(y_val, y_val_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# Output results
print(f"Random Forest Train Accuracy: {train_acc:.4f}")
print(f"Random Forest Validation Accuracy: {val_acc:.4f}")
print(f"Random Forest Test Accuracy: {test_acc:.4f}")

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Fit grid search to the training data
grid_search.fit(X_train_tfidf, y_train)

# Best parameters and model
print("Best Hyperparameters:", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_

# Evaluate tuned model
y_train_pred_tuned = best_rf_model.predict(X_train_tfidf)
y_val_pred_tuned = best_rf_model.predict(X_val_tfidf)
y_test_pred_tuned = best_rf_model.predict(X_test_tfidf)

# Accuracy calculations for tuned model
train_acc_tuned = accuracy_score(y_train, y_train_pred_tuned)
val_acc_tuned = accuracy_score(y_val, y_val_pred_tuned)
test_acc_tuned = accuracy_score(y_test, y_test_pred_tuned)

# Output tuned model results
print(f"TUNED Random Forest Train Accuracy: {train_acc_tuned:.4f}")
print(f"TUNED Random Forest Validation Accuracy: {val_acc_tuned:.4f}")
print(f"TUNED Random Forest Test Accuracy: {test_acc_tuned:.4f}")

SVM Train Accuracy: 0.7924
SVM Validation Accuracy: 0.7255
SVM Test Accuracy: 0.7256
Fitting 3 folds for each of 16 candidates, totalling 48 fits


KeyboardInterrupt: 

2) SVM

In [ ]:
# Split into train+validation and test sets first (80% train_val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Then split train+validation into train and validation sets (80/20 split of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)  # Final: 60% train, 20% val, 20% test

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Linear SVM Model
svm_model = LinearSVC(max_iter=10000, random_state=42)  # Increased max_iter for stability
svm_model.fit(X_train_tfidf, y_train)

# Predictions
y_train_pred = svm_model.predict(X_train_tfidf)
y_val_pred = svm_model.predict(X_val_tfidf)
y_test_pred = svm_model.predict(X_test_tfidf)

# Accuracy calculations
train_acc = accuracy_score(y_train, y_train_pred)
val_acc = accuracy_score(y_val, y_val_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# Output results
print(f"SVM Train Accuracy: {train_acc:.4f}")
print(f"SVM Validation Accuracy: {val_acc:.4f}")
print(f"SVM Test Accuracy: {test_acc:.4f}")

# Define parameter grid
svm_param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'loss': ['hinge', 'squared_hinge'],
    'max_iter': [20000, 30000]
}

# Set up GridSearchCV
svm_grid_search = GridSearchCV(
    LinearSVC(random_state=42),
    svm_param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Fit grid search to training data
svm_grid_search.fit(X_train_tfidf, y_train)

# Best parameters and model
print("Best SVM Hyperparameters:", svm_grid_search.best_params_)
best_svm_model = svm_grid_search.best_estimator_

# Predictions using the tuned model
y_train_pred_tuned = best_svm_model.predict(X_train_tfidf)
y_val_pred_tuned = best_svm_model.predict(X_val_tfidf)
y_test_pred_tuned = best_svm_model.predict(X_test_tfidf)

# Accuracy calculations for tuned model
train_acc_tuned = accuracy_score(y_train, y_train_pred_tuned)
val_acc_tuned = accuracy_score(y_val, y_val_pred_tuned)
test_acc_tuned = accuracy_score(y_test, y_test_pred_tuned)

# Output tuned model results
print(f"TUNED SVM Train Accuracy: {train_acc_tuned:.4f}")
print(f"TUNED SVM Validation Accuracy: {val_acc_tuned:.4f}")
print(f"TUNED SVM Test Accuracy: {test_acc_tuned:.4f}")

KeyboardInterrupt: 

3) Logistic Regression

In [ ]:
# Split into train+validation and test sets first (80% train_val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Then split train+validation into train and validation sets (80/20 split of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)  # 0.25 x 0.8 = 0.2, so final: 60% train, 20% val, 20% test

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Logistic Regression Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Predictions
y_train_pred = model.predict(X_train_tfidf)
y_val_pred = model.predict(X_val_tfidf)
y_test_pred = model.predict(X_test_tfidf)

# Accuracy calculations
train_acc = accuracy_score(y_train, y_train_pred)
val_acc = accuracy_score(y_val, y_val_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# Output results
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Hyperparameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs'],
    'max_iter': [1000]
}

# Grid search setup
grid_search = GridSearchCV(
    LogisticRegression(),
    param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Train grid search on training data
grid_search.fit(X_train_tfidf, y_train)

# Best parameters and model
print("Best Logistic Regression Parameters:", grid_search.best_params_)
best_lr_model = grid_search.best_estimator_

# Predictions using tuned model
y_train_pred_tuned = best_lr_model.predict(X_train_tfidf)
y_val_pred_tuned = best_lr_model.predict(X_val_tfidf)
y_test_pred_tuned = best_lr_model.predict(X_test_tfidf)

# Accuracy calculations for tuned model
train_acc_tuned = accuracy_score(y_train, y_train_pred_tuned)
val_acc_tuned = accuracy_score(y_val, y_val_pred_tuned)
test_acc_tuned = accuracy_score(y_test, y_test_pred_tuned)

# Output tuned model results
print(f"TUNED Logistic Regression Train Accuracy: {train_acc_tuned:.4f}")
print(f"TUNED Logistic Regression Validation Accuracy: {val_acc_tuned:.4f}")
print(f"TUNED Logistic Regression Test Accuracy: {test_acc_tuned:.4f}")

4) Naive Bayes

In [ ]:
# Split into train+validation and test sets first (80% train_val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Then split train+validation into train and validation sets (80/20 split of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)  # Final: 60% train, 20% val, 20% test

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Naive Bayes Model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Predictions
y_train_pred = nb_model.predict(X_train_tfidf)
y_val_pred = nb_model.predict(X_val_tfidf)
y_test_pred = nb_model.predict(X_test_tfidf)

# Accuracy calculations
train_acc = accuracy_score(y_train, y_train_pred)
val_acc = accuracy_score(y_val, y_val_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# Output results
print(f"Naive Bayes Train Accuracy: {train_acc:.4f}")
print(f"Naive Bayes Validation Accuracy: {val_acc:.4f}")
print(f"Naive Bayes Test Accuracy: {test_acc:.4f}")

# Parameter grid for Naive Bayes
nb_param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1.0, 2.0]  # Laplace smoothing
}

# Set up GridSearchCV
nb_grid_search = GridSearchCV(
    MultinomialNB(),
    nb_param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Fit grid search on training data
nb_grid_search.fit(X_train_tfidf, y_train)

# Best parameters and model
print("Best Naive Bayes Hyperparameters:", nb_grid_search.best_params_)
best_nb_model = nb_grid_search.best_estimator_

# Predictions using tuned model
y_train_pred_tuned = best_nb_model.predict(X_train_tfidf)
y_val_pred_tuned = best_nb_model.predict(X_val_tfidf)
y_test_pred_tuned = best_nb_model.predict(X_test_tfidf)

# Accuracy calculations for tuned model
train_acc_tuned = accuracy_score(y_train, y_train_pred_tuned)
val_acc_tuned = accuracy_score(y_val, y_val_pred_tuned)
test_acc_tuned = accuracy_score(y_test, y_test_pred_tuned)

# Output tuned model results
print(f"TUNED Naive Bayes Train Accuracy: {train_acc_tuned:.4f}")
print(f"TUNED Naive Bayes Validation Accuracy: {val_acc_tuned:.4f}")
print(f"TUNED Naive Bayes Test Accuracy: {test_acc_tuned:.4f}")